In [1]:
import pandas as pd
import os
import sys
from tqdm import tqdm
import re
import glob

In [16]:
# Make bed
def gtf_to_bed(gtf):
    bed_dict = {'chrom':[],
                'start':[],
                'stop':[],
                'name':[],
                'biotype':[],
                'strand':[]}

    for row in tqdm(gtf.itertuples(index=False)):
        if row._2 != 'gene':
            continue
        chrom = row._0
        start = row._3
        stop = row._4
        strand = row._6

    

        info = row._8
    
        biotype = re.search('gene_biotype "(.*?)";', info).group(1)

        try:
            gene_name = re.search('gene_name "(.*?)";', info).group(1)
        except AttributeError:
            gene_name = re.search('gene_id "(.*?)";', info).group(1)
    
        bed_dict['chrom'].append(chrom)
        bed_dict['start'].append(start)
        bed_dict['stop'].append(stop)
        bed_dict['name'].append(gene_name)
        bed_dict['biotype'].append(biotype)
        bed_dict['strand'].append(strand)
    bed = pd.DataFrame.from_dict(bed_dict)
    
    return bed


In [2]:
# Map ARS-UCD2.0 annotation to UOA_Wagyu_1.withY then overlap with
# UOA_Wagyu_1.withY annotation and see which genes are missing from ARS-UCD2.0

In [ ]:
gtfs = glob.glob('../gene_annotation_comparison/HER_mapped_to_WAG/*.gtf')

gtf_list = []

for gtf in gtfs:
    her_to_wag = pd.read_csv(gtf,
                             sep='\t', header=None,
                             comment='#',
                             dtype={0:str})
    
    bed = gtf_to_bed(her_to_wag)

    gtf_list.append(bed)

big_bed = pd.concat(gtf_list)
big_bed = big_bed.sort_values(by=['chrom','start'], ascending=True)
big_bed.to_csv('../gene_annotation_comparison/HER_mapped_to_WAG/HER.mapped_to.WAG.bed', sep='\t', header=None, index=None)

In [17]:
ang_to_wag = pd.read_csv('../gene_annotation_comparison/HER_mapped_to_WAG/ANG_Y.mappedTo.WAG_Y.gtf',
                         sep='\t', header=None,
                         comment='#',
                         dtype={0:str})
bed = gtf_to_bed(ang_to_wag)

bed.to_csv('../gene_annotation_comparison/HER_mapped_to_WAG/ANG_Y.mappedTo.WAG_Y.tmp', sep='\t', header=None, index=None)

3707it [00:00, 751342.66it/s]


In [ ]:
wag_gtf = pd.read_csv('../../../REFERENCES/UOA_WAGYU/UOA_Wagyu_1_Y.ensembl.gtf',
                      header=None, sep='\t', comment='#',
                      dtype={0:str})
wag_gtf

In [ ]:
wag_bed = gtf_to_bed(wag_gtf)
wag_bed.to_csv('../gene_annotation_comparison/HER_mapped_to_WAG/UOA_Wagyu_1_Y.ensembl.bed',
               sep='\t', header=None, index=None)

In [10]:
df = pd.read_csv('../gene_annotation_comparison/HER_mapped_to_WAG/WAG_genes.notIn.HER.bed',
                 sep='\t', header=None)
chroms_of_interest = [str(i) for i in range(1,30)]
chroms_of_interest.append('X')

df = df[df[0].isin(chroms_of_interest)].copy()
df[4].value_counts()

4
lncRNA                  7985
rRNA                    1340
protein_coding           738
miRNA                    163
TR_V_gene                 97
snoRNA                    82
snRNA                     66
pseudogene                31
IG_V_gene                 20
misc_RNA                  16
Y_RNA                     11
vault_RNA                  5
TR_J_gene                  3
ribozyme                   3
processed_pseudogene       3
IG_J_gene                  2
IG_D_gene                  1
Name: count, dtype: int64

In [15]:
df[(df[0] == 'X') & (df[4] == 'protein_coding')].sort_values(by=1).tail(20)

,0,1,2,3,4,5
13476,X,162757365,162759310,ENSBTAG00085028902,protein_coding,+
13483,X,164905492,164913366,AMELX,protein_coding,-
13484,X,164946070,165065224,ARHGAP6,protein_coding,+
13485,X,165063093,165071798,HCCS,protein_coding,-
13489,X,165193302,165193544,COX7B,protein_coding,-
13494,X,166288772,166290164,RFLNB,protein_coding,+
13506,X,170756974,170761626,ENSBTAG00085026004,protein_coding,-
13510,X,170940431,170949940,ENSBTAG00085026039,protein_coding,-
13516,X,172732535,172757857,ASMT,protein_coding,-
13517,X,172769861,172780781,AKAP17A,protein_coding,-


In [9]:
x = df.groupby(0)[4].value_counts().reset_index(name='count')
x.columns = ['chrom','biotype','count']
x.to_clipboard(index=False)

In [ ]:
df.shape

In [18]:
df = pd.read_csv('../gene_annotation_comparison/HER_mapped_to_WAG/WAG_genes.notIn.ANG.bed',
                 sep='\t', header=None)

df[4].value_counts()

4
protein_coding            215
pseudogene                 84
lncRNA                      9
tRNA                        5
transcribed_pseudogene      1
Name: count, dtype: int64

In [19]:
df

,0,1,2,3,4,5
0,Y,34576,50731,PLCXD1,protein_coding,+
1,Y,108635,112866,LOC132344458,lncRNA,+
2,Y,641983,709113,CSF2RA,protein_coding,+
3,Y,770056,818420,LOC132344462,protein_coding,-
4,Y,831192,840371,LOC112445918,protein_coding,-
...,...,...,...,...,...,...
309,Y,59356536,59357488,LOC132344567,pseudogene,-
310,Y,59362313,59373255,LOC132344536,protein_coding,+
311,Y,59380471,59381423,LOC132344568,pseudogene,-
312,Y,59383835,59397196,LOC132344537,protein_coding,+
